In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from art.defences.preprocessor.feature_squeezing import FeatureSqueezing

feature_squeezer = FeatureSqueezing(clip_values=(0, 1), bit_depth=1)

def preprocess(inputs, feature_squeezer):
    inputs_squeezed, _ = feature_squeezer(inputs)
    return torch.tensor(inputs_squeezed, dtype=torch.float32)


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/A2PM/preprocessor/FeatureSqueezing.csv", mode='a', index=False)




In [4]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test_tensor = preprocess(X_test, feature_squeezer).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/A2PM/preprocessor/FeatureSqueezing.csv", mode='a', index=False, header=False)




In [5]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_train_tensor = preprocess(x_train, feature_squeezer).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

# x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
x_val_tensor = preprocess(x_val, feature_squeezer).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [10]:
for epoch in range(50):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_train_loss = 0.0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            val_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = val_train_loss / len(val_loader)
    val_accuracy = correct_predictions / len(val_dataset)

    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping check using min_delta
    if best_loss - avg_val_loss > min_delta:
        best_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 0.0152, Validation Loss: 0.0013, Validation Accuracy: 0.9996
Epoch 2, Training Loss: 0.0012, Validation Loss: 0.0013, Validation Accuracy: 0.9996
Epoch 3, Training Loss: 0.0012, Validation Loss: 0.0013, Validation Accuracy: 0.9996
Epoch 4, Training Loss: 0.0012, Validation Loss: 0.0014, Validation Accuracy: 0.9996
Epoch 5, Training Loss: 0.0012, Validation Loss: 0.0013, Validation Accuracy: 0.9996
Epoch 6, Training Loss: 0.0012, Validation Loss: 0.0013, Validation Accuracy: 0.9996
Early stopping triggered


In [11]:
calculate_performance_metrics(x_test, y_test, model, 'DNN', 'Baseline', '0')

Accuracy: 0.9996649256763266

macro
Precision: 0.9909361334146615
Recall: 0.8949020138918613
F1 Score: 0.9346481931245328

weighted
Precision: 0.9996796610794622
Recall: 0.9996649256763266
F1 Score: 0.999660145624935

Mean FNR: 0.10509798610813872
Mean TNR: 0.9999324133820016
Mean FPR: 6.758661799841062e-05
Mean TPR: 0.8949020138918613


In [13]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start BIM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

print("start FGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

print("start PGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

print("start DF")
# DF_eps = [0.01, 0.1, 0.2, 0.3]
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)
# calculate_performance_metrics(x_test_adv, y_test_adv, model)

start BIM
Accuracy: 0.9976502913052402

macro
Precision: 0.7883947447127235
Recall: 0.8642366078671744
F1 Score: 0.764389824630322

weighted
Precision: 0.9986817172644604
Recall: 0.9976502913052402
F1 Score: 0.998105721116763

Mean FNR: 0.13576339213282568
Mean TNR: 0.9995283506317524
Mean FPR: 0.00047164936824750387
Mean TPR: 0.8642366078671744
Accuracy: 0.9894954199528383

macro
Precision: 0.6845322435913226
Recall: 0.7809398042745072
F1 Score: 0.6715910353586485

weighted
Precision: 0.9940531899018837
Recall: 0.9894954199528383
F1 Score: 0.991265205556604

Mean FNR: 0.21906019572549287
Mean TNR: 0.9978709949538409
Mean FPR: 0.002129005046159165
Mean TPR: 0.7809398042745072
Accuracy: 0.984008577902686

macro
Precision: 0.8409145487765389
Recall: 0.7701892009415165
F1 Score: 0.764528957548843

weighted
Precision: 0.9930951888056951
Recall: 0.984008577902686
F1 Score: 0.9870068913653792

Mean FNR: 0.22981079905848345
Mean TNR: 0.9967604706657303
Mean FPR: 0.0032395293342696203
Mean TPR

In [12]:
torch.save(model.state_dict(), "/home/jovyan/A2PM/preprocessor/dnn_pytorch_FeatureSqueezing.pt")

In [13]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BA', epsilon)

start AutoPGD
Accuracy: 0.99806075735174

macro
Precision: 0.7996510614501411
Recall: 0.792158184815204
F1 Score: 0.7115192957722597

weighted
Precision: 0.9991738705672375
Recall: 0.99806075735174
F1 Score: 0.9985211035434226

Mean FNR: 0.207841815184796
Mean TNR: 0.9996111600709195
Mean FPR: 0.0003888399290805579
Mean TPR: 0.792158184815204
Accuracy: 0.990169757029231

macro
Precision: 0.6704281509322098
Recall: 0.716856864419606
F1 Score: 0.615843454768998

weighted
Precision: 0.9926427333710742
Recall: 0.990169757029231
F1 Score: 0.9912847972087041

Mean FNR: 0.28314313558039395
Mean TNR: 0.9979924259597819
Mean FPR: 0.0020075740402181416
Mean TPR: 0.716856864419606
Accuracy: 0.9862158800098847

macro
Precision: 0.6489722325421615
Recall: 0.6738086080205721
F1 Score: 0.5650198187661254

weighted
Precision: 0.9915834523841256
Recall: 0.9862158800098847
F1 Score: 0.9884860238640648

Mean FNR: 0.32619139197942804
Mean TNR: 0.9971941050045425
Mean FPR: 0.002805894995457582
Mean TPR: 0.

In [15]:
filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_HopSkipJump_eps_0.npy'
x_test_adv = np.load(filename)

calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'HopSkipJump', '0')

Accuracy: 0.8165928805083077

macro
Precision: 0.3019347071082212
Recall: 0.5943081082488909
F1 Score: 0.3239785158267174

weighted
Precision: 0.9405406944488611
Recall: 0.8165928805083077
F1 Score: 0.8687990516399287

Mean FNR: 0.405691891751109
Mean TNR: 0.9344609354825375
Mean FPR: 0.0655390645174625
Mean TPR: 0.5943081082488909


In [ ]:
# epsilon_values = [0.01, 0.1, 0.2, 0.3]
# # Iterate over epsilon values
# print("start A2PM")
# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{epsilon}_fullattack_X.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

In [12]:
print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ZOO', epsilon)

start ZOO
Accuracy: 0.9993172860655154

macro
Precision: 0.8473760973041783
Recall: 0.8848291983578808
F1 Score: 0.8370318289941483

weighted
Precision: 0.999538033506635
Recall: 0.9993172860655154
F1 Score: 0.9994007160055038

Mean FNR: 0.1151708016421193
Mean TNR: 0.999862873226751
Mean FPR: 0.0001371267732489018
Mean TPR: 0.8848291983578808
Accuracy: 0.9993172860655154

macro
Precision: 0.8473760973041783
Recall: 0.8848291983578808
F1 Score: 0.8370318289941483

weighted
Precision: 0.999538033506635
Recall: 0.9993172860655154
F1 Score: 0.9994007160055038

Mean FNR: 0.1151708016421193
Mean TNR: 0.999862873226751
Mean FPR: 0.0001371267732489018
Mean TPR: 0.8848291983578808
Accuracy: 0.9993130976364695

macro
Precision: 0.8468840195137212
Recall: 0.8847078391345796
F1 Score: 0.8364426289084111

weighted
Precision: 0.9995374062071963
Recall: 0.9993130976364695
F1 Score: 0.9993980531755216

Mean FNR: 0.11529216086542027
Mean TNR: 0.9998620353935553
Mean FPR: 0.00013796460644469087
Mean TP